In [1]:
%cd ..
%load_ext autoreload
%autoreload 2

/home/teo/userdata/SejongMusic


In [2]:
from pathlib import Path
from music21 import converter, stream, note as m21_note 
import torch

from tqdm.auto import tqdm

from sejong_music.yeominrak_processing import AlignedScore, SamplingScore, pack_collate, ShiftedAlignedScore, OrchestraScore
from sejong_music.model_zoo import Seq2seq, Converter, AttentionSeq2seq, QkvAttnSeq2seq, get_emb_total_size, QkvAttnSeq2seqOrch
import random as random
from sejong_music.decode import MidiDecoder, OrchestraDecoder
import torch
from torch.utils.data import Dataset, DataLoader
from omegaconf import OmegaConf
from fractions import Fraction


/home/teo/.local/share/virtualenvs/SejongMusic-0Xd-sAL0/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

save_name = 'use_gugakwon'
state = torch.load('outputs/2024-03-07/11-03-56/epoch600_model.pt')
state_alt = torch.load('outputs/2024-03-07/11-03-56/best_loss_model.pt')

config = OmegaConf.load('yamls/orchestration.yaml')
config = get_emb_total_size(config)

output_dir = Path('gen_results/')
output_dir.mkdir(parents=True, exist_ok=True)

device = 'cpu'


In [12]:
val_dataset = OrchestraScore(is_valid= True,
                            # valid_measure_num = [i for i in range(93, 99)],
                            xml_path='music_score/FullScore_edit2.musicxml', 
                            use_pitch_modification=False, 
                            slice_measure_num=config.data.max_meas,
                            min_meas=config.data.min_meas,
                            feature_types=config.model.features,
                              sampling_rate=config.data.sampling_rate)



In [5]:
model = QkvAttnSeq2seqOrch(val_dataset.era_dataset.tokenizer, val_dataset.tokenizer, config.model).to(device)
model.is_condition_shifted = True
model.load_state_dict(state)
model.eval()


QkvAttnSeq2seqOrch(
  (encoder): Encoder(
    (emb): MultiEmbedding(
      (layers): ModuleList(
        (0): Embedding(8, 32)
        (1): Embedding(19, 64)
        (2): Embedding(16, 64)
        (3): Embedding(36, 32)
        (4): Embedding(7, 32)
        (5): Embedding(13, 32)
      )
    )
    (rnn): GRU(256, 128, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  )
  (decoder): QkvAttnDecoder(
    (emb): MultiEmbedding(
      (layers): ModuleList(
        (0): Embedding(5, 32)
        (1): Embedding(30, 64)
        (2): Embedding(26, 64)
        (3): Embedding(107, 32)
        (4): Embedding(7, 32)
        (5): Embedding(13, 32)
      )
    )
    (rnn): GRU(256, 128, num_layers=2, batch_first=True)
    (proj): Linear(in_features=256, out_features=56, bias=True)
    (query): Linear(in_features=128, out_features=128, bias=True)
    (key): Linear(in_features=128, out_features=128, bias=True)
    (value): Linear(in_features=128, out_features=128, bias=True)
  )
)

In [13]:
val_dataset.tokenizer.tok2idx['duration']

{'pad': 0,
 'start': 1,
 'end': 2,
 8.0: 21,
 0.25: 4,
 0.5: 5,
 0.75: 6,
 1.0: 7,
 1.25: 8,
 1.5: 9,
 2.0: 10,
 2.25: 11,
 2.5: 12,
 2.75: 13,
 3.0: 14,
 4.0: 15,
 4.25: 16,
 4.5: 17,
 5.5: 18,
 6.0: 19,
 7.5: 20,
 9.0: 22,
 11.5: 23}

In [ ]:
for i in range(30):
  offset = round_number(i/3, 3)
  if offset in test_set.tokenizer.tok2idx['offset']:
     continue
  if i < 6:
    other_value = i % 3 + 6
  else:
    other_value = i % 3 + 21
  test_set.tokenizer.tok2idx['offset'][offset] = test_set.tokenizer.tok2idx['offset'][round_number(other_value/3, 3)]

for i in range(30):
  offset = round_number(i/2, 3)
  if offset in test_set.tokenizer.tok2idx['offset']:
     continue
  other_value = i - 2
  test_set.tokenizer.tok2idx['offset'][offset] = test_set.tokenizer.tok2idx['offset'][round_number(other_value/2, 3)]

if 'offset' in model.tokenizer.tok2idx:
  for i in range(120):
    if i % 4 == 0:
      continue
    if i / 4 not in model.tokenizer.tok2idx['offset']:
      model.tokenizer.tok2idx['offset'][i/4] = model.tokenizer.tok2idx['offset'][(i-2)/4]


In [ ]:
  train_dataset = dataset_class(is_valid= False, 
                                xml_path=original_wd/'music_score/FullScore.musicxml',
                                use_pitch_modification = config.data.use_pitch_modification, 
                                pitch_modification_ratio=config.data.modification_ratio,
                                min_meas=config.data.min_meas, 
                                max_meas=config.data.max_meas,
                                feature_types=config.model.features,
                                sampling_rate=config.data.sampling_rate
                                )